### 1- Import

In [ ]:
%reload_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from process import *

### 2 - Fouille de données

In [ ]:
FILE = '../data/raw/cities.csv'
df = pd.read_csv(FILE)

In [ ]:
df.shape

In [ ]:
df.head(10)

In [ ]:
df['latitude'].isnull().sum(), df['longitude'].isnull().sum()

In [ ]:
df.dtypes

### Analyses
- id : identifiant
- local_name : triplé -> lieu, région, pays
- unique_name : nom du lieu
- latitude : coordonnée
- longitude	: coordonnée
- population : nombres d'habitants 

Plusieurs observations : 
- Certains lieux ne représentent pas la ville mais une localisation plus précise (exemple : ligne 23 'Aix En Provence Gare Routiere'). Si on tente d'automatiser la recherche du nombre d'habitants à partir de la colonne local_name ou unique_name, il se peut que l'on n'obtienne aucun résultat.
- De ce fait, il est plutot logique d'avoir autant de valeur NaN dans la colonne population.
- On remarque la présence de plusieurs langues / alphabets dans la colonne local_name. Il se peut qu'il y ait des doublons. (exemple : Barcelona, Barcelone)

### Data Preparation

In [ ]:
df_copy = df.copy()

In [ ]:
PRECISION = 4
index_duplicates = df_copy[df_copy.loc[:, ['latitude', 'longitude']].round(PRECISION).duplicated(keep=False)].index

In [ ]:
column_city = np.vectorize(get_city)(df_copy['latitude'], df_copy['longitude'])

In [ ]:
df_copy['city'] = column_city

In [ ]:
df_copy.loc[index_duplicates].sort_values('city')

Il y a bel et bien présence de doublons. Notre hypothèse sur les langues / alphabets est juste. <br/>
Par exemple, Bois-Le-Duc apparait 4 fois : <br/>
- En français
- En allemand
- En Néerlandais
- En Néerlandais avec une structure différente (la traduction a été ajoutée)

On pourrait choisir de supprimer les doublons. En contrepartie, il faudra fusionner les données dans les autres fichiers puisque qu'on suprrime des id.

In [ ]:
df_copy = df_copy.drop_duplicates(subset=['city'])

In [ ]:
percent_missing_before_merge = df_copy['population'].isnull().sum() * 100 / len(df_copy)

In [ ]:
column_population_unique_name = df_copy.apply(lambda x: get_population(x.unique_name) if pd.isnull(x.population) else x.population, axis=1)
column_population_city = df_copy.apply(lambda x: get_population(x.city) if pd.isnull(x.population) else x.population, axis=1)

In [ ]:
df_population = pd.DataFrame()
df_population['city_pop'] = column_population_city
df_population['unique_name_pop'] = column_population_unique_name

In [ ]:
df_population['city_pop'] = df_population['unique_name_pop'].where(pd.isnull(df_population['city_pop']),df_population['city_pop'])

In [ ]:
df_copy['population'] = df_population['city_pop'].where(pd.isnull(df_copy['population']),df_copy['population'])

In [ ]:
percent_missing_after_merge = df_copy['population'].isnull().sum() * 100 / len(df_copy)
print (f"pourcentage de valeurs manquantes dans la colonne population AVANT / APRES fusion: {percent_missing_before_merge} / {percent_missing_after_merge} %")

In [ ]:
df_copy.to_csv('../data/cleaned/cities_cleaned.csv')